In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
df = pd.read_csv("LosAngeles_Earthquake_Dataset.csv")
print("Dataset shape:", df.shape)
print(df.info())

Dataset shape: (22899, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22899 entries, 0 to 22898
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   latitude                        22899 non-null  float64
 1   longitude                       22899 non-null  float64
 2   mag                             22899 non-null  float64
 3   clustering_coefficient_30_days  22899 non-null  float64
 4   std_mag_30_days                 22899 non-null  float64
 5   rolling_mean_depth_30_days      22899 non-null  float64
 6   earthquakes_last_30_days        22899 non-null  int64  
 7   b_value                         22899 non-null  float64
 8   b_value_increment_i_i2          22899 non-null  float64
 9   b_value_increment_i2_i4         22899 non-null  float64
 10  b_value_increment_i4_i6         22899 non-null  float64
 11  b_value_increment_i6_i8         22899 non-null  float64
 12  b_val

In [78]:
feature_cols = [
    'latitude',
    'longitude',
    'mag',
    'clustering_coefficient_30_days',
    'std_mag_30_days',
    'rolling_mean_depth_30_days',
    'earthquakes_last_30_days',
    'b_value',
    'b_value_increment_i_i2',
    'b_value_increment_i2_i4',
    'b_value_increment_i4_i6',
    'b_value_increment_i6_i8',
    'b_value_increment_i8_i10',
    'max_mag_last_week',
    'eta',
    'delta_M',
    'elapsed_time',
    'coefficient_of_variation',
    'dE1_2'
]
target_col = 'class'


In [79]:
df = df[feature_cols + [target_col]].dropna()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[target_col])

X = df[feature_cols].values.astype(np.float32)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [81]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

In [82]:
class EarthquakeNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y))
model = EarthquakeNN(input_dim, hidden_dim, output_dim)

In [83]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 3500

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/3500], Loss: 1.7196
Epoch [20/3500], Loss: 1.6600
Epoch [30/3500], Loss: 1.6156
Epoch [40/3500], Loss: 1.5856
Epoch [50/3500], Loss: 1.5674
Epoch [60/3500], Loss: 1.5530
Epoch [70/3500], Loss: 1.5392
Epoch [80/3500], Loss: 1.5252
Epoch [90/3500], Loss: 1.5105
Epoch [100/3500], Loss: 1.4951
Epoch [110/3500], Loss: 1.4790
Epoch [120/3500], Loss: 1.4624
Epoch [130/3500], Loss: 1.4450
Epoch [140/3500], Loss: 1.4270
Epoch [150/3500], Loss: 1.4081
Epoch [160/3500], Loss: 1.3886
Epoch [170/3500], Loss: 1.3687
Epoch [180/3500], Loss: 1.3481
Epoch [190/3500], Loss: 1.3274
Epoch [200/3500], Loss: 1.3069
Epoch [210/3500], Loss: 1.2867
Epoch [220/3500], Loss: 1.2667
Epoch [230/3500], Loss: 1.2470
Epoch [240/3500], Loss: 1.2278
Epoch [250/3500], Loss: 1.2087
Epoch [260/3500], Loss: 1.1897
Epoch [270/3500], Loss: 1.1710
Epoch [280/3500], Loss: 1.1523
Epoch [290/3500], Loss: 1.1338
Epoch [300/3500], Loss: 1.1154
Epoch [310/3500], Loss: 1.0971
Epoch [320/3500], Loss: 1.0788
Epoch [330/3500],

In [84]:
model.eval()
with torch.no_grad():
    preds = model(X_test_t)
    predicted = torch.argmax(preds, dim=1).numpy()
target_names1 = [str(c) for c in label_encoder.classes_]

acc = accuracy_score(y_test, predicted)
print("\nAccuracy:", acc)
print("\nClassification report:\n", classification_report(y_test, predicted, target_names=target_names1))


Accuracy: 0.9150655021834061

Classification report:
               precision    recall  f1-score   support

           1       0.91      0.91      0.91       942
           2       0.92      0.92      0.92      1269
           3       0.93      0.91      0.92      1040
           4       0.93      0.93      0.93       687
           5       0.89      0.90      0.89       527
           6       0.94      0.90      0.92       115

    accuracy                           0.92      4580
   macro avg       0.92      0.91      0.91      4580
weighted avg       0.92      0.92      0.92      4580



In [85]:
class NumpyNN:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.001, seed=42):
        np.random.seed(seed)
        # Xavier initialization
        self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2 / (input_dim + hidden_dim))
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2 / (hidden_dim * 2))
        self.b2 = np.zeros((1, hidden_dim))
        self.W3 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2 / (hidden_dim + output_dim))
        self.b3 = np.zeros((1, output_dim))
        self.lr = lr

    def relu(self, x):
        return np.maximum(0, x)

    def drelu(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp / np.sum(exp, axis=1, keepdims=True)

    def forward(self, X):
        z1 = X @ self.W1 + self.b1
        a1 = self.relu(z1)
        z2 = a1 @ self.W2 + self.b2
        a2 = self.relu(z2)
        z3 = a2 @ self.W3 + self.b3
        a3 = self.softmax(z3)
        return (z1, a1, z2, a2, z3, a3)

    def compute_loss(self, y_true, y_pred):
        # Cross-entropy
        n = y_true.shape[0]
        log_probs = -np.log(y_pred[range(n), y_true] + 1e-8)
        return np.mean(log_probs)

    def backward(self, X, y_true, cache):
        z1, a1, z2, a2, z3, a3 = cache
        n = X.shape[0]

        # one-hot
        y_onehot = np.zeros_like(a3)
        y_onehot[np.arange(n), y_true] = 1

        dz3 = (a3 - y_onehot) / n
        dW3 = a2.T @ dz3
        db3 = np.sum(dz3, axis=0, keepdims=True)

        da2 = dz3 @ self.W3.T
        dz2 = da2 * self.drelu(z2)
        dW2 = a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)

        da1 = dz2 @ self.W2.T
        dz1 = da1 * self.drelu(z1)
        dW1 = X.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # gradient descent step
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3

    def train(self, X, y, epochs=100, batch_size=32):
        n = X.shape[0]
        losses = []
        for epoch in range(1, epochs + 1):
            idx = np.random.permutation(n)
            X = X[idx]
            y = y[idx]
            batch_loss = 0
            for i in range(0, n, batch_size):
                xb = X[i:i + batch_size]
                yb = y[i:i + batch_size]
                cache = self.forward(xb)
                loss = self.compute_loss(yb, cache[-1])
                batch_loss += loss * xb.shape[0]
                self.backward(xb, yb, cache)
            epoch_loss = batch_loss / n
            losses.append(epoch_loss)
            if epoch % 10 == 0 or epoch == 1:
                print(f"[NumPy NN] Epoch {epoch}/{epochs} - loss: {epoch_loss:.4f}")
        return losses

    def predict(self, X):
        _, _, _, _, _, a3 = self.forward(X)
        return np.argmax(a3, axis=1)


In [86]:
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y_train))

numpy_model = NumpyNN(input_dim, hidden_dim, output_dim, lr=0.001)
numpy_model.train(X_train, y_train, epochs=3500, batch_size=32)

[NumPy NN] Epoch 1/3500 - loss: 1.8065
[NumPy NN] Epoch 10/3500 - loss: 1.5996
[NumPy NN] Epoch 20/3500 - loss: 1.5643
[NumPy NN] Epoch 30/3500 - loss: 1.5408
[NumPy NN] Epoch 40/3500 - loss: 1.5221
[NumPy NN] Epoch 50/3500 - loss: 1.5062
[NumPy NN] Epoch 60/3500 - loss: 1.4915
[NumPy NN] Epoch 70/3500 - loss: 1.4773
[NumPy NN] Epoch 80/3500 - loss: 1.4632
[NumPy NN] Epoch 90/3500 - loss: 1.4491
[NumPy NN] Epoch 100/3500 - loss: 1.4351
[NumPy NN] Epoch 110/3500 - loss: 1.4209
[NumPy NN] Epoch 120/3500 - loss: 1.4067
[NumPy NN] Epoch 130/3500 - loss: 1.3923
[NumPy NN] Epoch 140/3500 - loss: 1.3778
[NumPy NN] Epoch 150/3500 - loss: 1.3632
[NumPy NN] Epoch 160/3500 - loss: 1.3484
[NumPy NN] Epoch 170/3500 - loss: 1.3336
[NumPy NN] Epoch 180/3500 - loss: 1.3189
[NumPy NN] Epoch 190/3500 - loss: 1.3042
[NumPy NN] Epoch 200/3500 - loss: 1.2898
[NumPy NN] Epoch 210/3500 - loss: 1.2752
[NumPy NN] Epoch 220/3500 - loss: 1.2606
[NumPy NN] Epoch 230/3500 - loss: 1.2461
[NumPy NN] Epoch 240/3500 -

[1.8065081819490179,
 1.7192379708177592,
 1.6790574449412903,
 1.6552295072035323,
 1.6392199943800176,
 1.6275777131080607,
 1.6185496475739445,
 1.6112117285250829,
 1.605013898136764,
 1.5996188576978214,
 1.5948045983521195,
 1.5904708646372379,
 1.586491512879967,
 1.5827952604197633,
 1.5793146634845305,
 1.5760287899677492,
 1.5729017384381807,
 1.5699116482313296,
 1.5670487382675684,
 1.5643034543654026,
 1.5616413732206602,
 1.5590654635793852,
 1.5565644509329506,
 1.5541375189664286,
 1.5517662207199303,
 1.5494624885214754,
 1.547244762353699,
 1.5450827182838136,
 1.5429533102504707,
 1.5408479399315262,
 1.5388266660326555,
 1.5368277897134712,
 1.5348735181133066,
 1.5329482540688482,
 1.531045149922974,
 1.529213463253896,
 1.527400844261906,
 1.5256336357310414,
 1.5238709004045632,
 1.5221483382537988,
 1.5204525134976272,
 1.5187611044590599,
 1.5171374918224958,
 1.5154737591003493,
 1.5139130035302895,
 1.5123218636528575,
 1.5107537931896582,
 1.5092025530366164

In [87]:
y_pred_np = numpy_model.predict(X_test)
acc_np = accuracy_score(y_test, y_pred_np)
print("\nNumPy NN Accuracy:", acc_np)
print("\nClassification report (NumPy NN):\n", classification_report(y_test, y_pred_np, target_names=target_names1))


NumPy NN Accuracy: 0.9091703056768559

Classification report (NumPy NN):
               precision    recall  f1-score   support

           1       0.92      0.90      0.91       942
           2       0.91      0.91      0.91      1269
           3       0.90      0.92      0.91      1040
           4       0.89      0.92      0.91       687
           5       0.90      0.90      0.90       527
           6       0.94      0.86      0.90       115

    accuracy                           0.91      4580
   macro avg       0.91      0.90      0.91      4580
weighted avg       0.91      0.91      0.91      4580

